In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
print(os.listdir("../input/citizens/Citizens"))

# Any results you write to the current directory are saved as output.

['citizens']
['datathon_propattributes.csv', 'datathon_propattributes.obj', 'demoNotebook.ipynb', 'demoNotebook.html', 'FileLayout.xlsx']


In [2]:
import pickle
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from scipy import stats

# You may not have this dependency downloaded! It cannot be installed traditionally with the pip command.
# You may have to create a python virtual environment to get this dependency. However, it is a visualization
# tool for the demo and you do not need it for your own project. Our suggestions is to view the demoNotebook.html
# file in browser to see the output of the demo.
import mpl_toolkits
 
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import Normalize    
%matplotlib inline
path = '../input/citizens/Citizens'

#Read in the data
test = open(path + '/datathon_propattributes.obj', 'rb')
test = pickle.load(test)

pd.set_option('display.max_rows', 200)

In [ ]:
#predict = test[test['transaction_dt'] > '2018-10-01']

In [3]:
col_use = ['prop_zip_code','construction_quality','condition','dwelling_type','acres','assessed_total_value','assessed_land_value','market_total_value','market_land_value','building_square_feet','transaction_dt', 'sale_amt']
new_test = pd.DataFrame(test[col_use])
col_value = ['acres','assessed_total_value','assessed_land_value','market_total_value','market_land_value','building_square_feet','sale_amt']
for col in col_value:
    new_test = new_test[(new_test[col]>0)]

In [4]:

new_test['acres'] = new_test['acres'].apply(lambda x: np.log(x))
new_test['market_total_value'] = new_test['market_total_value'].apply(lambda x: np.log(x))
new_test['market_land_value'] = new_test['market_land_value'].apply(lambda x: np.log(x))
new_test['assessed_total_value'] = new_test['assessed_total_value'].apply(lambda x: np.log(x))
new_test['assessed_land_value'] = new_test['assessed_land_value'].apply(lambda x: np.log(x))

new_test['building_square_feet'] = new_test['building_square_feet'].apply(lambda x: np.log(x))
new_test['sale_amt'] = new_test['sale_amt'].apply(lambda x: np.log(x))


In [5]:
for col in col_value:
    col_trim = [col]
    new_test = new_test[(np.abs(stats.zscore(new_test[col_trim])) < 3).all(axis=1)]

In [ ]:
h,x,y = np.histogram2d(new_test['acres'],new_test['sale_amt'])
plt.xlabel('acres')
plt.ylabel('sale_amt')
plt.imshow(h)

In [6]:
def var2num(test,var,n_label=0):
    if n_label==0:
        col = test[var].value_counts().index.values.tolist()
    else:
        col = test[var].value_counts().index[:n_label].values.tolist()
    ind = [i+1 for i in range(len(col))]
    Zip = zip(col,ind)
    dic = dict(Zip)
    new = test[var].apply(lambda x:dic[x] if x in dic else None)
    mean = new.mean()
    new_col = new.fillna(mean)
    return new_col

In [7]:
#var = 'prop_state'
#new_test[var] = var2num(new_test,var,n_label=0)

var = 'prop_zip_code'
new_test[var] = var2num(new_test,var,n_label=50)

var = 'construction_quality'
new_test[var] = var2num(new_test,var,n_label=0)

#var = 'roof_cover'
#new_test[var] = var2num(new_test,var,n_label=10)

var = 'condition'
new_test[var] = var2num(new_test,var,n_label=0)

var = 'dwelling_type'
new_test[var] = var2num(new_test,var,n_label=10)
len(new_test)

1421851

In [ ]:
h,x,y = np.histogram2d(new_test['prop_zip_code'],new_test['sale_amt'])
plt.xlabel('prop_zip_code')
plt.ylabel('sale_amt')
plt.imshow(h)

In [ ]:
h,x,y = np.histogram2d(new_test['construction_quality'],new_test['sale_amt'])
plt.xlabel('construction_quality')
plt.ylabel('sale_amt')
plt.imshow(h)

In [ ]:
h,x,y = np.histogram2d(new_test['condition'],new_test['sale_amt'])
plt.xlabel('condition')
plt.ylabel('sale_amt')
plt.imshow(h)

In [ ]:
h,x,y = np.histogram2d(new_test['dwelling_type'],new_test['sale_amt'])
plt.xlabel('dwelling_type')
plt.ylabel('sale_amt')
plt.imshow(h)

In [8]:
col_value = ['sale_amt','acres','assessed_total_value','assessed_land_value','market_total_value','market_land_value','building_square_feet']
#col_value.append('prop_zip_code')
col_value.append('transaction_dt')
new_test2 = pd.DataFrame(new_test[col_value])
new_test2.head()

,sale_amt,acres,assessed_total_value,assessed_land_value,market_total_value,market_land_value,building_square_feet,transaction_dt
0,11.589887,-0.478036,12.087289,10.905956,12.087289,10.905956,7.124478,2016-04-07
1,12.234417,-0.478036,12.087289,10.905956,12.087289,10.905956,7.124478,2017-07-12
2,7.685703,-0.478036,12.087289,10.905956,12.087289,10.905956,7.124478,2015-06-03
3,12.206073,-0.478036,12.087289,10.905956,12.087289,10.905956,7.124478,2008-06-30
4,11.773750,-2.120264,11.795092,9.957028,11.795092,9.957028,7.129298,2010-05-26


In [9]:
from sklearn.preprocessing import normalize

checkdate = '2018-10-01'
checkdate = pd.to_datetime(checkdate)
checkdateend = '2018-12-31'
checkdateend = pd.to_datetime(checkdateend)
testset = new_test[(new_test['transaction_dt'] >= checkdate)&(new_test['transaction_dt'] < checkdateend)]
trainset = new_test[new_test['transaction_dt'] < checkdate]


col_X = col_value [1:-1]
train_X = trainset[col_X]
train_Y = trainset['sale_amt']

test_X = testset[col_X]
test_Y = testset['sale_amt']
train_X = normalize(train_X, axis=0, norm='l1')
test_X = normalize(test_X, axis=0, norm='l1')

In [ ]:
print(np.shape(train_X),np.shape(test_X))

In [ ]:
##K-NN
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import RadiusNeighborsRegressor
print("\n")
print("[INFO] evaluating raw pixel accuracy...")
model = KNeighborsRegressor(n_neighbors=100, p = 2)
model.fit(train_X, np.array(train_Y))
acc = model.predict(test_X)
acc
pre = model.score(test_X, test_Y)
print("[INFO] SVM-SVC raw pixel accuracy: {:.2f}%".format(pre * 100))

train_X.head()

In [12]:
##Random Forest
from sklearn.ensemble import RandomForestRegressor
print("\n")
print("[INFO] evaluating raw pixel accuracy...")
model = RandomForestRegressor(n_estimators=30)
model.fit(train_X, np.array(train_Y))
acc = model.score(train_X, np.array(train_Y))
print("[INFO] Random Forest raw pixel accuracy: {:.2f}%".format(acc * 100))
acc = model.predict(test_X)
acc



[INFO] evaluating raw pixel accuracy...
[INFO] Random Forest raw pixel accuracy: 0.00%


array([11.59887043, 11.59887043, 11.59887043, ..., 11.70170743,
       11.59887043, 11.59887043])

In [ ]:
#SVC
from sklearn.svm import SVC
print("\n")
print("[INFO] evaluating raw pixel accuracy...")
model = SVC(max_iter=1000,class_weight='balanced',C = 2, gamma = 0.2)
model.fit(X_val, Y_val)
acc = model.score(X_val, Y_val)
print("[INFO] SVM-SVC raw pixel accuracy: {:.2f}%".format(acc * 100))

In [ ]:
##LGB
import lightgbm as lgb
import xgboost as xgb
def run_lgb(train_X, train_y, val_X, val_y, test_X):
    params = {
        "objective" : "regression",
        "metric" : "rmse",
        'boosting_type' : 'goss',
        'max_depth' : 6,#-1
        "num_leaves" : 20,#20
        "learning_rate" : 0.001,#0.01
        #"bagging_fraction" : 0.6,#0.7 #0.8 #0.3
        "feature_fraction" : 0.7,#0.7 #0.5
        #"bagging_freq" : 2, #10 #20
        "bagging_seed" : 42, #2018
        "verbosity" : -1,
        'lambda_l2' : 0.0001,#0.1
        'lambda_l1' : 0.001,#0,
        'max_bin' : 300 #default=250 #200 #170 #120 #90
    }
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    evals_result = {}
    model = lgb.train(params, lgtrain, 1000, valid_sets=[lgtrain, lgval], early_stopping_rounds=10000, 
                      verbose_eval=200, evals_result=evals_result)
    
    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    return pred_test_y, model, evals_result

In [ ]:
pred ,_, evals_result =run_lgb(train_x, train_y, val_x1, val_y1, test_X)

In [13]:
pererror = []
counts = 0
labels = list(test_Y)
for i in range(len(acc)):
    pererror.append((acc[i]-labels[i])/labels[i])
    if np.abs(pererror[i]) < 0.1:
        counts += 1
counts/len(acc)


0.8207309998066138

In [ ]:
model.predict(train_X)

In [ ]:
np.std(acc) / np.mean(acc) , np.min(acc),np.max(acc)


In [ ]:
plt.hist(acc,bins = 50)
plt.show()

In [ ]:
pererror = []
counts = 0
labels = list(test_Y)
for i in range(len(acc)):
    pererror.append((acc[i]-labels[i])/labels[i])
    if np.abs(pererror[i]) < 0.1:
        counts += 1
counts/len(acc)


In [ ]:
###LGB
import lightgbm as lgb
import xgboost as xgb
def run_lgb(train_X, train_y, val_X, val_y, test_X):
    params = {
        "objective" : "regression",
        "metric" : "rmse",
        'boosting_type' : 'goss',
        'max_depth' : 6,#-1
        "num_leaves" : 20,#20
        "learning_rate" : 0.001,#0.01
        #"bagging_fraction" : 0.6,#0.7 #0.8 #0.3
        "feature_fraction" : 0.7,#0.7 #0.5
        #"bagging_freq" : 2, #10 #20
        "bagging_seed" : 42, #2018
        "verbosity" : -1,
        'lambda_l2' : 0.0001,#0.1
        'lambda_l1' : 0.001,#0,
        'max_bin' : 300 #default=250 #200 #170 #120 #90
    }
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    evals_result = {}
    model = lgb.train(params, lgtrain, 1000, valid_sets=[lgtrain, lgval], early_stopping_rounds=10000, 
                      verbose_eval=200, evals_result=evals_result)
    
    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    return pred_test_y, model, evals_result

In [ ]:
pred ,_, evals_result =run_lgb(train_x, train_y, val_x1, val_y1, test_X)

In [ ]:

new_test = new_test[(np.abs(stats.zscore(new_test[['sale_amt']])) < 3).all(axis=1)]
h,x,y = np.histogram2d(new_test['building_square_feet'],new_test['sale_amt'])

plt.imshow(h)

In [ ]:
new_test = new_test[(np.abs(stats.zscore(new_test[['market_total_value']])) < 3).all(axis=1)]
new_test = new_test[(np.abs(stats.zscore(new_test[['sale_amt']])) < 3).all(axis=1)]
h,x,y = np.histogram2d(new_test['market_total_value'],new_test['sale_amt'])

plt.imshow(h)

In [ ]:
new_test = new_test[(np.abs(stats.zscore(new_test[['market_land_value']])) < 3).all(axis=1)]
new_test = new_test[(np.abs(stats.zscore(new_test[['sale_amt']])) < 3).all(axis=1)]
h,x,y = np.histogram2d(new_test['market_land_value'],new_test['sale_amt'])

plt.imshow(h)

In [ ]:
pd.set_option("display.colheader_justify", 'left')
test2=pd.read_excel(path + '/FileLayout.xlsx')
pd.set_option('display.max_rows', 72)
pd.set_option('max_colwidth', 500)
test2[21:42]

In [ ]:
total =len(test['dwelling_type']) 
train1 = pd.DataFrame(test['dwelling_type'].value_counts()/total)
train1['index'] = 1
for i in range(total):
    if train1.[] 
  #      train1[index] = 0
train1


In [ ]:
test['construction_quality'].unique()
test['construction_quality'].value_counts()

In [ ]:
checkdate = '2018-10-01'
checkdate = pd.to_datetime(checkdate)
testset = test[test['transaction_dt'] >= checkdate]


In [ ]:
for col in test.columns:
    print(test[col].value_counts(dropna=False).head(3))

In [ ]:
import matplotlib.pyplot as plt

plt.hist(test['transaction_dt'],bins = 10)
plt.show()

In [ ]:
for col in test.columns[21:40]:
    print(col)

In [ ]:
plt.hist(test.columns[21],bins = 10)
plt.show()

In [ ]:
var = test.columns[21]
test[var]
test['sale_amt']

In [ ]:
var = test.columns[22]
plt.scatter(test[var],test['sale_amt'])
plt.xlabel(var)
plt.ylabel('sale_amt')
plt.xlim(0, 200)
plt.show()


In [ ]:
import seaborn as sns
from scipy.stats import norm, skew
#test['sale_amt'] = np.log1p(test['sale_amt'])

#Check the new distribution 
sns.distplot(test['sale_amt'] , fit=norm);


In [ ]:
test[var].sort_values()

In [ ]:

data = pd.concat([test[var], test['sale_amt']], axis=1)
data = data[(data[var] != 0) & (data['sale_amt'] != 0)]
data[var].sort_values()

In [ ]:
sns.set()
data = pd.concat([test[test.columns[21]],test[test.columns[22]] test['sale_amt']], axis=1)
cols = [test.columns[21], test.columns[22], 'sale_amt']
sns.pairplot(test[cols], height = 2.5)
plt.show();

In [ ]:
from sklearn.linear_model import Ridge,Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

data  = pd.concat([test[test.columns[21]], test[test.columns[22]]], axis=1)
data = data[(data[test.columns[21]] != 0) & (data[test.columns[22]] != 0)]
x_plot = data.iloc[:,0]
model = make_pipeline(PolynomialFeatures(1), LinearRegression())
model.fit(X_plot, data.iloc[:,1])
mse = mean_squared_error(model.predict(X_plot), data.iloc[:,1])
y_plot = model.predict(x_plot[:, np.newaxis])
plt.plot(x_plot, y_plot, color='red',
                 linewidth=2)

plt.scatter(x_plot, data.iloc[:,1], color='navy', s=30, marker='o', label="training points")
#poly = PolynomialFeatures(degree=2)
#model.fit(X_plot, data.iloc[:,1])
#y_plot = model.predict(X_plot)
#plt.plot(X_plot, y_plot, linewidth=2, label="degree %d" % 2)
plt.show()

In [ ]:
from scipy.optimize import minimize
data  = pd.concat([test[test.columns[21]], test[test.columns[22]]], axis=1)
data = data[(data[test.columns[21]] != 0) & (data[test.columns[22]] != 0)]
x_plot = data.iloc[:,0]
X_plot = x_plot[:, np.newaxis]
def fit(X, params):
    return X*params[0]+X*params[1]

def cost_function(params, X, y):
    return np.sum(np.abs(y - fit(X, params)))

output = minimize(cost_function, [1,1], args=(X_plot, data.iloc[:,1]))
y_hat = fit(np.c_[data[:,0], data[:,1]], output.x)
y_plot = fit(X, output.x)

plt.scatter(x_plot, data.iloc[:,1], color='navy', s=30, marker='o', label="training points")
plt.plot(x_plot, data.iloc[:,1], color='red',linewidth=2)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.cbook import boxplot_stats
import pandas as pd
import seaborn as sns

data = [
    ('LA', 1),
    ('Sultan', 128),
    ('ElderCare', 1),
    ('CA', 3),
    ('More', 900),
]
df = pd.DataFrame(data, columns=('client', 'total'))
ax = sns.boxplot(data=test)
outliers = [y for stat in boxplot_stats(test[]) for y in stat['fliers']]
print(outliers)
for y in outliers:
    ax.plot(1, y, 'p')
ax.set_xlim(right=1.5)
plt.show()

In [ ]:

test[test.columns[23]].unique()


In [ ]:
import seaborn as sns
sns.set()
data = pd.concat([test[test.columns[24]],test[test.columns[25]],test[test.columns[26]], test['sale_amt']], axis=1)
cols = [test.columns[24], test.columns[25], test.columns[26],'sale_amt']
sns.pairplot(test[cols], height = 2.5)
plt.show();

In [ ]:
sns.set()
data = pd.concat([test[test.columns[27]],test[test.columns[28]],test[test.columns[29]], test['sale_amt']], axis=1)
cols = [test.columns[27], test.columns[28], test.columns[29],'sale_amt']
sns.pairplot(test[cols], height = 2.5)
plt.show();

In [ ]:
test1 = test[test[test.columns[30]] > 0]

plt.hist(np.log(test1[test1.columns[30]]), bins = 20)
plt.show()

In [ ]:
sns.set()
data = pd.concat([test[test.columns[30]],test[test.columns[31]], test['sale_amt']], axis=1)
data = data[(data['tax_amt']>0) & (data['tax_year']>0) & (data['tax_amt']<np.exp(12))]
cols = [data.columns[0],data.columns[1],'sale_amt']
sns.pairplot(data[cols], height = 2.5)
plt.show();

In [ ]:
test[test.columns[33]].unique()

In [ ]:
sns.set()
data = pd.concat([test[test.columns[32]],test[test.columns[33]], test['sale_amt']], axis=1)
data = data[(data['assessed_year']>0)]
cols = [data.columns[0],data.columns[1],'sale_amt']
sns.pairplot(data[cols], height = 2.5)
plt.show();

In [ ]:
sns.set()
data = pd.concat([test['building_square_feet'],test['total_living_square_feet'],test['total_ground_floor_square_feet'],test['total_basement_square_feet'] ,test['total_garage_parking_square_feet'],test['sale_amt']], axis=1)
data = data[(data['building_square_feet']>0) & (data['total_living_square_feet']>0) & (data['total_ground_floor_square_feet']>0)&(data['total_ground_floor_square_feet']<5*10000) & (data['total_basement_square_feet']<10000) & (data['total_garage_parking_square_feet']<10000)&(data['sale_amt']<4*100000000)]
cols = ['building_square_feet','total_living_square_feet','total_ground_floor_square_feet','total_basement_square_feet','total_garage_parking_square_feet','sale_amt']
sns.pairplot(data[cols], height = 2.5)
plt.show();

In [ ]:
data = pd.concat([test['building_square_feet'],test['sale_amt']],axis = 1)
data = data[data['building_square_feet']>0]
plt.scatter(np.log(data['building_square_feet']),np.log(data['sale_amt']))
plt.show()
        

In [ ]:
var = test.columns[21]
data = pd.concat([test[var], test['sale_amt']], axis=1)
data = data[(data[var] != 0) & (data['sale_amt'] != 0)]
data[var].sort_values()

sns.distplot(np.log(data[var]));

In [ ]:
sns.distplot(np.log(data['sale_amt']));

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

x_plot = np.log(data[var])
X_plot = x_plot[:, np.newaxis]
plt.scatter(x_plot, np.log(data['sale_amt']), color='navy', s=30, marker='o', label="training points")
model = make_pipeline(PolynomialFeatures(1), Ridge())
model.fit(X_plot, np.log(data['sale_amt']))
y_plot = model.predict(X_plot)
plt.plot(X_plot, y_plot, linewidth=2, label="degree %d" % 2)
plt.show()

In [ ]:
##Random Forest
from sklearn.ensemble import RandomForestClassifier
print("\n")
print("[INFO] evaluating raw pixel accuracy...")
model = RandomForestClassifier(n_estimators=200)
model.fit(X_train, y_train)
acc = model.score(X_test, y_test)
print("[INFO] Random Forest raw pixel accuracy: {:.2f}%".format(acc * 100))
y_predict = model.predict(X_test)
print(confusion_matrix(y_test, y_predict))
print('\n')
print(classification_report(y_test,y_predict))